In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

import math

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWi2a23g4GrQ8mEJlOZHqMpxTyTX6Rwb3Z_9p7jTMRNg8pESkeiEyK4



Successfully saved authorization token.


### Imports and Constants

In [11]:
import ee
import geemap

Map = geemap.Map()

# Emily's Areas
region1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]])
region2 = ee.Geometry.Polygon(
        [[[25.85, -10.6],
          [25.85, -10.7],
          [25.95, -10.7],
          [25.95, -10.6]]])
region3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]])
region4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]])
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]])
total = ee.Geometry.Polygon(
        [[[25.8, -10.5],
          [25.8, -10.8],
          [26.25, -10.8],
          [26.25, -10.5]]])

# Rishi Area
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])
# Ray Areas 
roi1 = ee.Geometry.Polygon(
        [[[29.55, 3.16],
          [29.55, 3.1],
          [29.63, 3.1],
          [29.63, 3.16]]])
roi2 = ee.Geometry.Polygon(
        [[[29.76, 3.16],
          [29.76, 3.12],
          [29.8, 3.12],
          [29.8, 3.16]]])
roi3 = ee.Geometry.Polygon(
        [[[30.25, 1.8],
          [30.25, 1.7],
          [30.35, 1.7],
          [30.35, 1.8]]])

# Sentinel-2 Data and Visualization
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

### Classifier

### Select Desired Region

In [13]:
mainRegion = region1

### Create Composite Image of Entire Region (Sentinel-2)

In [16]:
imageRef = s2 \
        .filter(ee.Filter.bounds(mainRegion)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B4', 'B3', 'B2') \
        .median().clip(mainRegion)
Map.centerObject(mainRegion, 11)
Map.addLayer(imageRef, rgbVis, 'export region whole')
Map

Map(bottom=278230.0, center=[-10.775008323246253, 25.825000000002206], controls=(WidgetControl(options=['posit…

### Create Square Regions

In [6]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

In [7]:
regions = create_segments(mainRegion, 5)
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 8


In [20]:
num = 1
for region in regions:
    imageRef = s2 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B4', 'B3', 'B2') \
        .median().clip(region)
    task = ee.batch.Export.image.toDrive(image=imageRef,
                                         description='passing region ' + str(num),
                                         folder='ee multi export test',
                                         scale=30,
                                         region=region,
                                         fileNamePrefix='region_ex_' + str(num),
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF')
    task.start()
    num = num+1

In [26]:
print(task.status())

{'state': 'READY', 'description': 'passing region 8', 'creation_timestamp_ms': 1645199455270, 'update_timestamp_ms': 1645199455270, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'E6X722YMX3WZJZENMEHVZ37K', 'name': 'projects/earthengine-legacy/operations/E6X722YMX3WZJZENMEHVZ37K'}


In [4]:
# export big region
imageRef = s2 \
        .filter(ee.Filter.bounds(total)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B4', 'B3', 'B2') \
        .median().clip(total)
task = ee.batch.Export.image.toDrive(image=imageRef,
                                     description='Big Region',
                                     folder='ee entire region',
                                     scale=30,
                                     region=total,
                                     fileNamePrefix='big_region',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [9]:
print(task.status())

{'state': 'COMPLETED', 'description': 'Big Region', 'creation_timestamp_ms': 1645638551710, 'update_timestamp_ms': 1645638748626, 'start_timestamp_ms': 1645638631498, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1zwXiaAYuNQuBL8u2eQQIPco2WlvA2IqW'], 'attempt': 1, 'id': '5V22KHNUZQJB5JTUCTNH377L', 'name': 'projects/earthengine-legacy/operations/5V22KHNUZQJB5JTUCTNH377L'}


In [10]:
Map.addLayer(imageRef, rgbVis, 'bigger region')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…